In [1]:
#import

import pandas
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.preprocessing import StandardScaler  # to standardize the features
from sklearn.decomposition import PCA
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import scipy.spatial
from textblob import TextBlob
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from google.colab import drive
import glob



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:

module_url = "https://tfhub.dev/google/universal-sentence-encoder/4" #@param ["https://tfhub.dev/google/universal-sentence-encoder/4", "https://tfhub.dev/google/universal-sentence-encoder-large/5"]
model = hub.load(module_url)
print ("module %s loaded" % module_url)

module https://tfhub.dev/google/universal-sentence-encoder/4 loaded


In [3]:


def processText(column,n):#to remove stop words,to convert into lowercase and root word
    import re

    corpus = []
    for i in range(n):
        review = re.sub('[^a-zA-Z]', ' ', column[i])
        review = review.lower()
        review = review.split()
        ps = PorterStemmer()
        all_stopwords = stopwords.words('english')
        all_stopwords.remove('not')
        review = [ps.stem(word) for word in review if not word in set(all_stopwords)]
        review = ' '.join(review)
        corpus.append(review)
    return corpus


In [4]:
def embed(input):
  return model(input)

In [5]:
def drop_2023data(df):
  # print(df['datetime'])
  df=df.dropna()
  # print("pal")
  df = df[~(df['datetime'] >= '30-12-2022')]#dropping rows of year=2023

  df=df.reset_index(drop=True)

  # print("udrasht")
  
  # print(df['datetime'])
  return df



In [6]:
def find_sentment(corpus):
  sentimentColumns=[]
  n=0
  p=0
  for content in corpus:
    blob = TextBlob(content)
    sentiment=blob.sentiment.polarity
    if sentiment<0:
      sentimentColumns.append(0)
      n=n+1
    else:
      sentimentColumns.append(1)
      p=p+1
  print(n,p,"sentement")

  return sentimentColumns


In [7]:
def find_y_data(df):
  n,m=df.shape
  daily = df['close'].shift(-1)/df['close'] - 1
  y = np.zeros(n)
  for i in range(n):
    if daily[i] >= 0:
      y[i] = 1 # up
    else:
      y[i] = 0 # down
  print(len(y))
  # y = np.insert(y, 0, 0)
  y=y[:-1]
  print(len(y))
  headers = ['y_lables']
  y_df = pd.DataFrame(y, columns=headers)
  y_df['datetime']=df['datetime']
  return y_df


In [8]:
def normalize_data(df):
  scalar = StandardScaler()
  scaled_df = scalar.fit_transform(df) #scaling the csv data
  df=scaled_df
  return df

In [9]:
def apply_PCA(titleColumn,fild="0"):
  text_vector=[]
  for text in titleColumn:
      sentence = text
      message_embeddings = embed([sentence])
      # message_embedding_snippet = ", ".join((str(x) for x in message_embeddings[:3]))
      temp=np.array(message_embeddings[0])
      temp=temp.T
      text_vector.append(temp)
  text_vector=np.array(text_vector)
  scalar = StandardScaler()
  scaled_data = scalar.fit_transform(text_vector)
  pca = PCA(n_components = 30)
  pca.fit(scaled_data)
  data_pca = pca.transform(scaled_data)
  print("pca")
  df_after_pca=pd.DataFrame(data_pca, columns = ['pca1'+fild,'pca2'+fild,'pca3'+fild,'pca4'+fild,'pca5'+fild,'pca6'+fild,'pca7'+fild,'pca8'+fild,'pca9'+fild,'pca10'+fild,'pca11'+fild,'pca12'+fild,'pca13'+fild,'pca14'+fild,'pca15'+fild,'pca16'+fild,'pca17'+fild,'pca18'+fild,'pca19'+fild,'pca20'+fild,'pca21'+fild,'pca22'+fild,'pca23'+fild,'pca24'+fild,'pca25'+fild,'pca26'+fild,'pca27'+fild,'pca28'+fild,'pca29'+fild,'pca30'+fild])
  print("pca2")
  return df_after_pca





  
def concat_data(df,df_after_pca,dateColumn,sentementvalue,df_after_pca_summery,sentementvalue_of_summary):
  column=['close', 'open', 'low', 'high', 'volume', 'evm',  'force_index', 'rsi', 'cci', 'macd-signal', 'atr', 'vwap','stx_7_3',
  'Total Revenue', 'Cost of Revenue', 'Gross Profit', 'Normalized EBITDA',  'Operating Cash Flow', 'Investing Cash Flow', 'Financing Cash Flow',
   'Free Cash Flow', 'Total Assets',  'Total Liabilities Net Minority Interest', 'Total Equity Gross Minority Interest', 'Total Capitalization']
  # column=['close', 'open', 'low', 'high', 'volume', 'evm',  'force_index', 'rsi', 'cci', 'macd-signal', 'atr', 'vwap']
 
  # print(df[0])
  df=pd.DataFrame(df,columns=column)
  # print(df)
  new_data=pd.concat([dateColumn,df,df_after_pca,df_after_pca_summery],axis=1)
  new_data['Sentiment']=sentementvalue
  new_data['sentiment_summary']=sentementvalue_of_summary
  return new_data


In [10]:
def divide_data(new_data,y_data):
  x_train = new_data[(new_data['datetime'] <'2022-01-01')]
  x_test = new_data[(new_data['datetime'] >='2022-01-01')]
  y_train = y_data[(y_data['datetime'] <'2022-01-01')]
  y_test = y_data[(y_data['datetime'] >='2022-01-01')]
  print("train x: ",x_train.shape,"train y: ",y_train.shape)
  print("test x: ",x_test.shape,"test y: ",y_test.shape)
# 2022-01-01

  #remove datetime column from data
  x_train = x_train.drop('datetime', axis=1)
  # file_train = file_train.drop('title', axis=1)
  x_test = x_test.drop('datetime', axis=1)
  y_train = y_train.drop('datetime', axis=1)
  y_test = y_test.drop('datetime', axis=1)
  return x_train,x_test,y_train,y_test


  

In [11]:
def ML_Model(x_train,x_test,y_train,y_test):
  # x_train['Volume'] = np.log(x_train['Volume']+0.5)
  # x_test['Volume'] = np.log(x_test['Volume']+0.5)
  baseline_LR = LogisticRegression(C = 1e10, tol=0.000000001, max_iter=100000)
  baseline_LR.fit(x_train, y_train)
      # print('Coefficients: \n', baseline_LR.coef_)
  print("training accracy: %.4f" % baseline_LR.score(x_train, y_train))
  print("test accuracy: %.4f" % baseline_LR.score(x_test, y_test))
  y_fit_LR = baseline_LR.predict(x_test)
  # print(y_fit_LR)
  # print(y_df_test)
  print('Accuracy:', accuracy_score(y_test, y_fit_LR))
  print('ROC AUC Score:', roc_auc_score(y_test, y_fit_LR))
  print('F1 score:', f1_score(y_test, y_fit_LR))
  print()
  print('\n clasification report:\n', classification_report(y_test, y_fit_LR))
  print('\n confussion matrix:\n',confusion_matrix(y_test, y_fit_LR))
  

In [12]:


# Set the directory path
dir_path = 'data/'

# Get a list of all files in the directory
file_list = os.listdir(dir_path)

# Filter only CSV files
csv_files = [file for file in file_list if file.endswith('.csv')]
for item in csv_files:
    #read each file from data folder
    df = pandas.read_csv(dir_path+"/"+item)
  #   df = df.drop(['stx_7_3',
  # 'Total Revenue', 'Cost of Revenue', 'Gross Profit', 'Normalized EBITDA',  'Operating Cash Flow', 'Investing Cash Flow', 'Financing Cash Flow',
  #  'Free Cash Flow', 'Total Assets',  'Total Liabilities Net Minority Interest', 'Total Equity Gross Minority Interest', 'Total Capitalization'], axis=1)
    # df['datetime'] = pd.to_datetime(df['datetime'], format='%d-%m-%Y')
    #remove 2023 data from dataset

    df=drop_2023data(df)
    y_data=pd.DataFrame(columns=['datetime','y_lable'])
    y_data['datetime']=df['datetime']
    y_data['y_lable']=df['y_actual']
    df=df.drop(['y_actual'],axis='columns')

    #take title column value to apply Sentiment analysis and Pca
    # take datetime column  value to create y lables of perticular data
    titleColumn=df['title']
    dateColumn=df['datetime']
    summarycolumn=df['summary']

    #preprocessing title column
    corpus=processText(titleColumn,len(titleColumn))
    df['title']=corpus
    df = df.astype({'title':'string'})
    sentementvalue=find_sentment(corpus)

    #preprocessing title column
    corpus_summary=processText(summarycolumn,len(summarycolumn))
    df['summary']=corpus_summary
    df = df.astype({'summary':'string'})
    sentementvalue_of_summary=find_sentment(corpus_summary)

  
    # df=pd.get_dummies(df,columns=['sentiment'],drop_first=True)
    
    #create y lables for data 
    # y_data = find_y_data(df)

    #dropping the date,title column from the data
    print(df.shape)
    df=df.drop(['datetime','title','summary'],axis='columns')
    df=normalize_data(df)

    df_after_pca=apply_PCA(titleColumn,"title")
    df_after_pca_summery=apply_PCA(summarycolumn,"summary")
   
    
    #concate news pca data with the ofiginal data
    new_data=concat_data(df,df_after_pca,dateColumn,sentementvalue,df_after_pca_summery,sentementvalue_of_summary)

    #divide data in test train

    x_train,x_test,y_train,y_test=divide_data(new_data,y_data)
    #Apply Model
    ML_Model(x_train,x_test,y_train,y_test)
  








72 902 sentement
145 829 sentement
(974, 28)
pca
pca2
pca
pca2
train x:  (724, 88) train y:  (724, 2)
test x:  (250, 88) test y:  (250, 2)


/usr/local/lib/python3.9/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


training accracy: 0.6533
test accuracy: 0.4640
Accuracy: 0.464
ROC AUC Score: 0.4797349459598559
F1 score: 0.5472972972972974


 clasification report:
               precision    recall  f1-score   support

         0.0       0.50      0.26      0.34       134
         1.0       0.45      0.70      0.55       116

    accuracy                           0.46       250
   macro avg       0.47      0.48      0.45       250
weighted avg       0.48      0.46      0.44       250


 confussion matrix:
 [[35 99]
 [35 81]]
49 925 sentement
22 952 sentement
(974, 28)
pca
pca2
pca
pca2
train x:  (724, 88) train y:  (724, 2)
test x:  (250, 88) test y:  (250, 2)


/usr/local/lib/python3.9/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


training accracy: 0.6340
test accuracy: 0.5320
Accuracy: 0.532
ROC AUC Score: 0.5005141057772636
F1 score: 0.016806722689075633


 clasification report:
               precision    recall  f1-score   support

         0.0       0.53      0.99      0.69       133
         1.0       0.50      0.01      0.02       117

    accuracy                           0.53       250
   macro avg       0.52      0.50      0.35       250
weighted avg       0.52      0.53      0.38       250


 confussion matrix:
 [[132   1]
 [116   1]]


In [13]:


# df = pandas.read_csv('AAPL_combine.csv')


# df=df.dropna()
# df = df[~(df['datetime'] >= '2023-01-01')]#dropping rows of year=2023

# df=df.reset_index(drop=True)


# print(df.columns)

# titleColumn=df['title']
# dateColumn=df['datetime']
# titleColumn=list(titleColumn)

# corpus=processText(titleColumn,len(titleColumn))#preprocessing title column
# df['title']=corpus

# df = df.astype({'title':'string'})
# print(df.tail(5))

In [14]:
# df.shape

In [15]:
# titleColumn=df['title']
# titleColumn[0]

# # df=df.fillna(0)

# # df=df.dropna()

In [16]:
# df.head(10)

In [17]:
# from textblob import TextBlob

In [18]:
# sentimentColumns=[]
# for content in corpus:
#     blob = TextBlob(content)
#     sentiment=blob.sentiment.polarity
#     if sentiment<0:
#         sentimentColumns.append("negative")
#     else:
#         sentimentColumns.append("positive")

# df['sentiment']=sentimentColumns

In [19]:
# print(df.head(5))

In [20]:
# df=pd.get_dummies(df,columns=['sentiment'],drop_first=True)

In [21]:
# df.columns

In [22]:
# df['sentiment_positive'].value_counts()

In [23]:
# print(df.head(5))

In [24]:
# n,m=df.shape
# daily = df['close'].shift(-1)/df['close'] - 1
# y = np.zeros(n)
# for i in range(n):
#   if daily[i] >= 0:
#     y[i] = 1 # up
#   else:
#     y[i] = 0 # down
# print(len(y))
# y = np.insert(y, 0, 0)
# y=y[:-1]
# print(len(y))
# headers = ['y_lables']
# y_df = pd.DataFrame(y, columns=headers)
# y_df['datetime']=df['datetime']
# y_df.head(10)



In [25]:
# df.tail(5)

In [26]:
# # df_title=df[['datetime','title']]
# df=df.drop(['datetime','title'],axis='columns')#dropping the date column


In [27]:
# scalar = StandardScaler()
# scaled_df = scalar.fit_transform(df) #scaling the csv data

# df=scaled_df


In [28]:
# df.shape

In [29]:

# text_vector=[]# will contain vector representation of titleColumn

In [30]:
# # text_vector=[]
# for text in titleColumn:
#     sentence = text
#     message_embeddings = embed([sentence])
#     # message_embedding_snippet = ", ".join((str(x) for x in message_embeddings[:3]))
#     temp=np.array(message_embeddings[0])
#     temp=temp.T
#     text_vector.append(temp)


In [31]:
# text_vector=np.array(text_vector)

In [32]:
# text_vector.shape

In [33]:
# # text_vector=np.array(text_vector)
# scalar = StandardScaler()
# scaled_data = scalar.fit_transform(text_vector) #scaling the data


In [34]:
# pca = PCA(n_components = 25)
# pca.fit(scaled_data)
# data_pca = pca.transform(scaled_data)
# data_pca

In [35]:
# data_pca.shape

In [36]:
# text_vector_dataframe=pd.DataFrame()
# type(data_pca)
# # df_news  =pd.DataFrame(((x,) for x in data_pca), columns=['title'])
# df_news=pd.DataFrame(data_pca, columns = ['pca1','pca2','pca3','pca4','pca5','pca6','pca7','pca8','pca9','pca10','pca11','pca12','pca13','pca14','pca15','pca16','pca17','pca18','pca19','pca20','pca21','pca22','pca23','pca24','pca25'])

In [37]:
# df_news

In [38]:
# text_vector_dataframe.describe()
# text_vector_dataframe['title']=data_pca

In [39]:
# column=['close', 'open', 'low', 'high', 'volume', 'evm',  'force_index', 'rsi', 'cci', 'macd-signal', 'atr', 'vwap', 'stx_7_3',
#   'Total Revenue', 'Cost of Revenue', 'Gross Profit', 'Normalized EBITDA',  'Operating Cash Flow', 'Investing Cash Flow', 'Financing Cash Flow',
#    'Free Cash Flow', 'Total Assets',  'Total Liabilities Net Minority Interest', 'Total Equity Gross Minority Interest', 'Total Capitalization','Sentiment']

# df=pd.DataFrame(df,columns=column)
# newDF=pd.concat([dateColumn,df,df_news],axis=1)

In [40]:
# newDF.shape

In [41]:
# newDF

In [42]:
# newDF.columns

Start applying model after this.

Take data before 2022-9-30 as training and after as testing.


In [43]:
# file_train = newDF[(newDF['datetime'] <'2022-01-01')]
# file_test = newDF[(newDF['datetime'] >='2022-01-01')]
# y_df_train = y_df[(y_df['datetime'] <'2022-01-01')]
# y_df_test = y_df[(y_df['datetime'] >='2022-01-01')]
# print("train x: ",file_train.shape,"train y: ",y_df_train.shape)
# print("test x: ",file_test.shape,"test y: ",y_df_test.shape)



In [44]:
# file_train = file_train.drop('datetime', axis=1)
# # file_train = file_train.drop('title', axis=1)
# file_test = file_test.drop('datetime', axis=1)
# y_df_train = y_df_train.drop('datetime', axis=1)
# y_df_test = y_df_test.drop('datetime', axis=1)
# file_train

In [45]:
# file_test

In [46]:
# y_df_train

In [47]:
# y_df_test

In [48]:
# # file_train.columns = file_train.iloc[0]
# file_train = file_train.iloc[1:]

# # file_test.columns = file_test.iloc[0]
# file_test = file_test.iloc[1:]


# # y_df_train.columns = y_df_train.iloc[0]
# y_df_train = y_df_train.iloc[1:]


# # y_df_test.columns = y_df_test.iloc[0]
# y_df_test = y_df_test.iloc[1:]

In [49]:
# print(y_df_test)

In [50]:
# from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import accuracy_score
# from sklearn.metrics import roc_auc_score
# from sklearn.metrics import f1_score
# from sklearn.metrics import confusion_matrix
# from sklearn.metrics import classification_report




# baseline_LR = LogisticRegression(C = 1e10, tol=0.000000001, max_iter=100000)
# baseline_LR.fit(file_train, y_df_train)
#     # print('Coefficients: \n', baseline_LR.coef_)
# print("training accracy: %.4f" % baseline_LR.score(file_train, y_df_train))
# print("test accuracy: %.4f" % baseline_LR.score(file_test, y_df_test))
# y_fit_LR = baseline_LR.predict(file_test)
# # print(y_fit_LR)
# # print(y_df_test)
# print('Accuracy:', accuracy_score(y_df_test, y_fit_LR))
# print('ROC AUC Score:', roc_auc_score(y_df_test, y_fit_LR))
# print('F1 score:', f1_score(y_df_test, y_fit_LR))
# print()
# print('\n clasification report:\n', classification_report(y_df_test, y_fit_LR))
# print('\n confussion matrix:\n',confusion_matrix(y_df_test, y_fit_LR))

In [51]:
# from sklearn.linear_model import LogisticRegression
# from sklearn import svm
# from sklearn.metrics import accuracy_score
# from sklearn.metrics import roc_auc_score
# from sklearn.metrics import f1_score
# from sklearn.metrics import confusion_matrix
# from sklearn.metrics import classification_report
# def calc_accuracy(pred, y):
#     pred = np.matrix(pred).reshape(-1, 1)
#     y = np.matrix(y).reshape(-1, 1)
#     count = y.shape[0]
#     correct = 0
#     for i in range (count):
#         if pred[i,0] == y[i, 0]:
#             correct += 1
#     return (correct * 1.0 / count)
# baseline_LR = LogisticRegression(C = 1e10, tol=0.000000001, max_iter=100000)
# baseline_LR.fit(file_train, y_df_train)
# # Training acc
# pred = baseline_LR.predict(file_train)
# accuracy = calc_accuracy(pred, y_df_train)
# print('The training accuracy of the Logistic Regression is ', accuracy)
# # Test acc
# y_hat = baseline_LR.predict(file_test)
# test_accuracy = calc_accuracy(y_hat, y_df_test)
# print('The test accuracy of the Logistic Regression is ', test_accuracy)
# # Conf matrix
# print('\n Training clasification report:\n', classification_report(y_df_train, pred))
# print('\n confusion matrix:\n',confusion_matrix(y_df_train, pred))
# print('\n Test clasification report:\n', classification_report(y_df_test, y_hat))
# print('\n confusion matrix:\n',confusion_matrix(y_df_test, y_hat))

# # Fit and report SVM
# # c = 0.5
# svm_rbf = svm.SVC(C=0.5, kernel='rbf',max_iter=50000)
# svm_rbf.fit(file_train, y_df_train)
# # Training acc
# pred = svm_rbf.predict(file_train)
# accuracy = calc_accuracy(pred, y_df_train)
# print('The training accuracy of the SVM is ', accuracy)
# # Test acc
# y_hat = svm_rbf.predict(file_test)
# test_accuracy = calc_accuracy(y_hat, y_df_test)
# print('The test accuracy of the SVM is ', test_accuracy)
# # Conf matrix
# print('\n Training classification report:\n', classification_report(y_df_train, pred))
# print('\n confusion matrix:\n',confusion_matrix(y_df_train, pred))
# print('\n Test classification report:\n', classification_report(y_df_test, y_hat))
# print('\n confusion matrix:\n',confusion_matrix(y_df_test, y_hat))